In [2]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import csv
from Bio.SeqRecord import SeqRecord
import torch
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
import itertools
from collections import Counter

In [3]:
from transformers import T5Tokenizer, T5EncoderModel, T5ForConditionalGeneration
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

2024-04-21 20:50:41.426569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 20:50:41.454254: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 20:50:41.588127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 20:50:41.588181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 20:50:41.611241: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

cpu


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('Rostlab/ProstT5', do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/ProstT5").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:


# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/ProstT5', do_lower_case=False).to(device)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/ProstT5").to(device)

# only GPUs support half-precision currently; if you want to run on CPU use full-precision (not recommended, much slower)
model.full() if device=='cpu' else model.half()

# prepare your protein sequences/structures as a list.
# Amino acid sequences are expected to be upper-case ("PRTEINO" below)
# while 3Di-sequences need to be lower-case ("strctr" below).
sequence_examples = ["PRTEINO", "strct"]

# replace all rare/ambiguous amino acids by X (3Di sequences do not have those) and introduce white-space between all sequences (AAs and 3Di)
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]

# The direction of the translation is indicated by two special tokens:
# if you go from AAs to 3Di (or if you want to embed AAs), you need to prepend "<AA2fold>"
# if you go from 3Di to AAs (or if you want to embed 3Di), you need to prepend "<fold2AA>"
sequence_examples = [ "<AA2fold>" + " " + s if s.isupper() else "<fold2AA>" + " " + s # this expects 3Di sequences to be already lower-case
                      for s in sequence_examples
                    ]

# tokenize sequences and pad up to the longest sequence in the batch
ids = tokenizer.batch_encode_plus(sequences_example,
                                  add_special_tokens=True,
                                  padding="longest",
                                  return_tensors='pt').to(device)

# generate embeddings
with torch.no_grad():
    embedding_rpr = model(
              ids.input_ids, 
              attention_mask=ids.attention_mask
              )

# extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens, incl. prefix ([0,1:8]) 
emb_0 = embedding_repr.last_hidden_state[0,1:8] # shape (7 x 1024)
# same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:6])
emb_1 = embedding_repr.last_hidden_state[1,1:6] # shape (5 x 1024)

# if you want to derive a single representation (per-protein embedding) for the whole protein
emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)

2024-04-21 20:25:48.263252: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 20:25:48.265227: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 20:25:48.292804: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 20:25:48.292825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 20:25:48.293595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: 'T5Tokenizer' object has no attribute 'to'

In [18]:
training = pd.read_csv('training_deepfri.csv',index_col=0)
training

,Sequence,Label,Protein
0,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,1,seq1|1|training
1,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,1,seq2|1|training
2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,1,seq3|1|training
3,MARVGPGRAGVSCQGRGRGRGGSGQRRPPTWEISDSDAEDSAGSEA...,1,seq4|1|training
4,MAAADGGGPGGASVGTEEDGGGVGHRTVYLFDRREKESELGDRPLQ...,1,seq5|1|training
...,...,...,...
2513,MEQGKGLAVLILAIILLQGTLAQSIKGNHLVKVYDYQEDGSVLLTC...,0,seq2516|0|training
2514,MARGPGLAPPPLRLPLLLLVLAAVTGHTAAQDNCTCPTNKMTVCSP...,0,seq2517|0|training
2515,MELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGL...,0,seq2518|0|training
2516,MWCIVLFSLLAWVYAEPTMYGEILSPNYPQAYPSEVEKSWDIEVPE...,0,seq2519|0|training


In [19]:
testing = pd.read_csv('testing_deepfri.csv',index_col=0)
testing

,Sequence,Label,Protein
0,MAELKSLSGDAYLALSHGYAAAAAGLAYGAAREPEAARGYGTPGPG...,1,sp|Q8NDY6|BHE23_HUMAN
1,MPDPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSIYVYKVL...,1,sp|P23527|H2B1O_HUMAN
2,MYQSLALAASPRQAAYADSGSFLHAPGAGSPMFVPPARVPSMLSYL...,1,sp|Q9BWX5|GATA5_HUMAN
3,MNMEGLVMFQDLSIDFSQEEWECLDAAQKDLYRDVMMENYSSLVSL...,1,sp|A8MQ14|ZN850_HUMAN
4,MTRSCSAVGCSTRDTVLSRERGLSFHQFPTDTIQRSKWIRAVNRVD...,1,sp|Q9H5L6|THAP9_HUMAN
...,...,...,...
625,MPLAQLADPWQKMAVESPSDSAENGQQIMDEPMGEEEINPQTEEVS...,0,sp|P51812|KS6A3_HUMAN
626,MAILMLSLQLILLLIPSISHEAHKTSLSSWKHDQDWANVSNMTFSN...,0,sp|Q6UY13|YB003_HUMAN
627,MAQASPPRPERVLGASSPEARPAQEALLLPTGVFQVAEKMEKRTCA...,0,sp|Q9UIL8|PHF11_HUMAN
628,MGKSEGPVGMVESAGRAGQKRPGFLEGGLLLLLLLVTAALVALGVL...,0,sp|Q495T6|MMEL1_HUMAN
